In [1]:
import numpy as np
import scipy as sp
import scipy.linalg
import cvxpy as cp
import random

In [2]:
def NKron(*args):
  result = np.array([[1.0]])
  for op in args:
    result = np.kron(result, op)
  return result

In [3]:
Id = np.eye(2)
X = np.array([[0.0, 1.0],[1.0, 0.0]])
Z = np.array([[1.0, 0.0],[0.0, -1.0]])
Y = np.matmul(X,Z)

In [4]:
NormalizeState = lambda state: state / sp.linalg.norm(state)
zero = np.array([[1.0], [0.0]]) # |0>
one = np.array([[0.0], [1.0]]) # |1>

## Generators

In [5]:
def NKronModified(checkRowMod):
  result = np.array([[1.0]])
  for ind in checkRowMod:
    if(ind == 0):
        op = Id
    elif(ind == 1):
        op = X
    elif(ind == 2):
        op = Y
    elif(ind == 3):
        op = Z
    result = np.kron(result, op)
  return result

def getGenerator(checkRow):
    checkRowModified = np.zeros(n, dtype=int)
    
    checkRowModified[(checkRow[:n] == checkRow[n:]) & (checkRow[n:] == 1)] = 2
    checkRowModified[(checkRow[:n] == 1) & (checkRowModified != 2)] = 1
    checkRowModified[(checkRow[n:] == 1) & (checkRowModified != 2)] = 3
    
    return NKronModified(checkRowModified)    

In [6]:
comparingAccuracy_decoded = 1e-7
comparingAccuracy_encoded = 1e-5
comparingAccuracy_syndrome = 1e-5
comparingAccuracy_method = 1e-5

In [7]:
# change check matrix here

# checkMatrix = np.array([[1,1,1,1,1,1,1,1, 0,0,0,0,0,0,0,0],
#                         [0,0,0,0,0,0,0,0, 1,1,1,1,1,1,1,1],
#                         [0,1,0,1,1,0,1,0, 0,0,0,0,1,1,1,1],
#                         [0,1,0,1,0,1,0,1, 0,0,1,1,0,0,1,1],
#                         [0,1,1,0,1,0,0,1, 0,1,0,1,0,1,0,1]])

# checkMatrix = np.array([[0,0,0,0,0,0,0,0,0, 1,1,0,0,0,0,0,0,0],
#                         [0,0,0,0,0,0,0,0,0, 1,0,1,0,0,0,0,0,0],
#                         [0,0,0,0,0,0,0,0,0, 0,0,0,1,1,0,0,0,0],
#                         [0,0,0,0,0,0,0,0,0, 0,0,0,1,0,1,0,0,0],
#                         [0,0,0,0,0,0,0,0,0, 0,0,0,0,0,0,1,1,0],
#                         [0,0,0,0,0,0,0,0,0, 0,0,0,0,0,0,1,0,1],
#                         [1,1,1,1,1,1,0,0,0, 0,0,0,0,0,0,0,0,0],
#                         [1,1,1,0,0,0,1,1,1, 0,0,0,0,0,0,0,0,0]])

checkMatrix = np.array([[1,0,0,1,0, 0,1,1,0,0],
                        [0,1,0,0,1, 0,0,1,1,0],
                        [1,0,1,0,0, 0,0,0,1,1],
                        [0,1,0,1,0, 1,0,0,0,1]])

# checkMatrix = np.array([[0,0,0,1,1,1,1, 0,0,0,0,0,0,0],
#                         [0,1,1,0,0,1,1, 0,0,0,0,0,0,0],
#                         [1,0,1,0,1,0,1, 0,0,0,0,0,0,0],
#                         [0,0,0,0,0,0,0, 0,0,0,1,1,1,1],
#                         [0,0,0,0,0,0,0, 0,1,1,0,0,1,1],
#                         [0,0,0,0,0,0,0, 1,0,1,0,1,0,1]])

n = int(checkMatrix.shape[1]/2)
k = n-checkMatrix.shape[0]

gi = np.zeros([n-k, 2**n, 2**n])
for i in range(n-k):
    gi[i,:,:] = getGenerator(checkMatrix[i,:])

## Encoding

In [8]:
def NKron1DGeneral(ipArray):
    result = np.array([[1.0]])
    for i in ipArray:
        if(i==1):
            op = one
        elif(i==0):
            op = zero
        result = np.kron(result, op)
    return result

#### Get generator matrix G

In [9]:
Gmatrix = np.eye(gi[0,:,:].shape[0], gi[0,:,:].shape[1]) # generator matrix corresponding to this code
for i in range(n-k):
    Gmatrix = Gmatrix + np.matmul(gi[i,:,:], Gmatrix)
Gmatrix = np.round(Gmatrix)

#### Get non-zero and unique columns of G

In [10]:
# get boolean array if the columns are zero or not
zeroCols = np.zeros(Gmatrix.shape[1])
for i in range(Gmatrix.shape[1]):
    zeroCols[i] = all(Gmatrix[:,i] == np.zeros(Gmatrix.shape[0]))

# get indices of non-zero columns
nonZeroColsList = np.argwhere(zeroCols==0).flatten()

# get all non zero columns
GmatrixNonZero = np.zeros([Gmatrix.shape[0], nonZeroColsList.shape[0]])
i = 0
for ind in nonZeroColsList:
    GmatrixNonZero[:,i] = Gmatrix[:,ind]
    i = i+1

# get all non zero and unique columns and there indices
GmatrixNonZeroUniqueInd, nonZeroUniqueInd = np.unique(GmatrixNonZero, axis = 1, return_index=True)
nonZeroUniqueInd = nonZeroColsList[nonZeroUniqueInd]

In [11]:
print('Rank of G = ' + str(np.linalg.matrix_rank(Gmatrix)))
print('Shape of G = ' + str(Gmatrix.shape))

Rank of G = 2
Shape of G = (32, 32)


#### Generate random transmit qbits

In [12]:
# generate qbits randomly
tx_qbits = np.random.rand(2**k)
tx_qbits = NormalizeState(tx_qbits)
print('Randomly generated qbit(s) = ' + str(tx_qbits.flatten()))

Randomly generated qbit(s) = [0.45192137 0.89205778]


#### Convert qbits to tensor product format

___Method 2___

More straightforward way.

In [13]:
tx_decoded = np.zeros(2**n)
A = np.eye(2**n, 2**n) # condition on codes

# get extended qbits corresponding to non-zero column indices of G matrix
i = 0
for nonZeroIndex in np.sort(nonZeroUniqueInd):
    if(i>=2**k):
        break
    tx_decoded[nonZeroIndex] = tx_qbits[i]
    
    A[nonZeroIndex, nonZeroIndex] = 0
    i = i+1
tx_decoded = NormalizeState(tx_decoded)

In [14]:
# print('Are the two methods giving same vector? ' + str( np.sum( np.abs(tx_decoded - tx_decoded2) > comparingAccuracy_method ) == 0))

#### Encode using generators

In [15]:
tx_encoded = NormalizeState(tx_decoded) # encoded transmit qbits
for i in range(n-k):
    tx_encoded = tx_encoded + np.matmul(gi[i,:,:], tx_encoded) # encode using generators
tx_encoded = NormalizeState(tx_encoded) # encoded transmit qbits

## Channel

In [16]:
p_xyz = 0.15 # p_xyz < 1/3
p_channel = [1-3*p_xyz, p_xyz, p_xyz, p_xyz] 
errMatrix = np.random.multinomial(1, p_channel, size=n)

errCheckRowModified = errMatrix@np.array([0,1,2,3])

channel_error = NKronModified(errCheckRowModified) # channel error
# channel_error = getGenerator(np.array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]))
rx_erry = np.dot(channel_error, tx_encoded) # received qbits with errors

In [17]:
strError = np.array([])

for errVal in errCheckRowModified:
    if(errVal == 0):
        strError = np.append(strError, 'I')
    if(errVal == 1):
        strError = np.append(strError, 'X')
    if(errVal == 2):
        strError = np.append(strError, 'Y')
    if(errVal == 3):
        strError = np.append(strError, 'Z')

print('Channel = ' + str(errCheckRowModified) + ' = ' + str(strError))

Channel = [3 0 0 0 0] = ['Z' 'I' 'I' 'I' 'I']


## Decoding

#### Syndrome Check

In [18]:
syndr = np.zeros([n-k, 1]) # syndrome
for i in range(n-k):
    syndr[i] = np.dot(rx_erry.transpose(), np.dot(gi[i,:,:], rx_erry))
print('Syndrome = ' + str( np.ndarray.astype( np.round(syndr.flatten()), int) ))

Syndrome = [-1  1 -1  1]


In [19]:
def getSyndromeFromError(channel_error):
    tx_qbits = np.ones(2**k)
    tx_qbits = NormalizeState(tx_qbits)

    # Convert qbits to tensor product format
    tx_decoded = np.zeros(2**n)
    # get extended qbits corresponding to non-zero column indices of G matrix
    i = 0
    for nonZeroIndex in np.sort(nonZeroUniqueInd):
        if(i>=2**k):
            break
        tx_decoded[nonZeroIndex] = tx_qbits[i]
        i = i+1
    tx_decoded = NormalizeState(tx_decoded)

    # encode transmit qbits
    tx_encoded = NormalizeState(tx_decoded) # encoded transmit qbits
    for i in range(n-k):
        tx_encoded = tx_encoded + np.matmul(gi[i,:,:], tx_encoded) # encode using generators
    tx_encoded = NormalizeState(tx_encoded) # encoded transmit qbits

    # channel
    rx_erry = np.dot(channel_error, tx_encoded) # received qbits with errors

    # syndrome check
    syndr = np.zeros([n-k, 1]) # syndrome
    for i in range(n-k):
        syndr[i] = np.dot(rx_erry.transpose(), np.dot(gi[i,:,:], rx_erry))
    
    syndr[syndr>0] = 0
    syndr[syndr<0] = 1
    
    return syndr

In [20]:
getSyndromeFromError(channel_error)

array([[1.],
       [0.],
       [1.],
       [0.]])

#### Error Correction

In [21]:
def getCardin(myVector):
    return np.sum(myVector != 0)

def getErrorFromSyndrome(syndr):
    success = 0
    finalError = np.zeros(2*errCheckRowModified.shape[0])
    while(getCardin(syndr) != 0):
        maxMetric = 0
        for generatorInd in range(n-k): # for all generators
            g = checkMatrix[generatorInd, :] # get the genrator
            g_modified = np.zeros(n, dtype=int)
            g_modified[(g[:n] == g[n:]) & (g[n:] == 1)] = 2
            g_modified[(g[:n] == 1) & (g_modified != 2)] = 1
            g_modified[(g[n:] == 1) & (g_modified != 2)] = 3
            
            string_format = '{:0>' + str(2*getCardin(g_modified)) + '}'
            for errorIndex in range(2**(2*getCardin(g_modified))): # for all errors with the support of that generator
                if(errorIndex == 0): continue
                thisError = np.copy(g_modified)
                
                modifyError = list(string_format.format("{:b}".format(errorIndex)))
                modifyError =  np.asarray(list(map(int, modifyError)) )
                
                temp_n = getCardin(g_modified)
                modifyErrorModified = np.zeros(temp_n, dtype=int)
                modifyErrorModified[(modifyError[:temp_n] == modifyError[temp_n:]) & (modifyError[temp_n:] == 1)] = 2
                modifyErrorModified[(modifyError[:temp_n] == 1) & (modifyErrorModified != 2)] = 1
                modifyErrorModified[(modifyError[temp_n:] == 1) & (modifyErrorModified != 2)] = 3
                
                thisError[thisError != 0] = modifyErrorModified # get the error
                
                           
                thisError1 = np.copy(thisError)
                thisError1[thisError == 1] = 1
                thisError1[thisError == 2] = 1
                thisError1[thisError == 3] = 0
                
                thisError2 = np.copy(thisError)
                thisError2[thisError == 1] = 0
                thisError2[thisError == 2] = 1
                thisError2[thisError == 3] = 1
                
                thisError = np.append(thisError1, thisError2)
                
                syndr_new = (syndr + getSyndromeFromError(getGenerator(thisError)))%2 # update to syndrome to check weight
                thisMetric = (getCardin(syndr) - getCardin(syndr_new))/getCardin(modifyErrorModified) # get the metric
                
                if(thisMetric > maxMetric):
                    bestError = thisError
                    maxMetric = thisMetric
#                 if(thisMetric == maxMetric):
#                     print('Error = ' + str(thisError) + ', |s_i+1| = ' + str(getCardin(syndr_new)) + ', |s_i| = ' + str(getCardin(syndr)) + ', |e| = ' + str(getCardin(thisError)))

        if(maxMetric != 0):
            finalError = bestError
            syndr = (syndr + getSyndromeFromError(getGenerator(bestError)))%2
        if(maxMetric == 0):
            break        
#         print('Max metric = ' + str(maxMetric) + ', Best error = ' + str(bestError) + ', Syndrome = ' + str(syndr))

    if(getCardin(syndr) != 0): success = 0
    else: success = 1

    return finalError.flatten(), success

In [22]:
# syndrome lookup table
def SyndromeLookUp(syndr):
    syndr[syndr>0] = 0
    syndr[syndr<0] = 1
    
    error, success = getErrorFromSyndrome(syndr)
    recov = getGenerator(error)
    return recov

In [23]:
recov = SyndromeLookUp(syndr) # error recovery
rx_encoded = np.matmul(recov.transpose(), rx_erry) # received qbits without error but still encoded

print('Number of qubits in error before error correction = ' + str(np.sum(np.abs(rx_erry - tx_encoded) > comparingAccuracy_encoded)))
print('Number of qubits in error after error correction = ' + str(np.sum(np.abs(rx_encoded - tx_encoded) > comparingAccuracy_encoded)))
print('MSE after error correction = ' + str(np.sum((rx_encoded - tx_encoded)**2)/np.sum(tx_encoded!=0) ))

Number of qubits in error before error correction = 16
Number of qubits in error after error correction = 0
MSE after error correction = 0.0


#### Complete Decoding

In [24]:
# setup optimizer to decode
P = np.matmul(Gmatrix.transpose(), Gmatrix)
q = -np.matmul(rx_encoded.transpose(), Gmatrix).flatten()
x = cp.Variable(rx_encoded.shape[0])

# get qbit that is at closest distance to received encoded qbit
prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, P) + q.T@x), [A@x == np.zeros(x.shape[0])])
prob.solve()
rx_decoded = NormalizeState(x.value) # received decoded qbits

print('Transmitted qbit = ' + str(tx_decoded[abs(tx_decoded) > comparingAccuracy_encoded]))
print('Decoded qbit = ' + str(rx_decoded[abs(rx_decoded) > comparingAccuracy_decoded]))
print('')

error = tx_decoded - rx_decoded
error = np.sum(error**2)/np.sum(np.abs(tx_decoded) > comparingAccuracy_decoded)
print('MSE = ' + str(error))

Transmitted qbit = [0.45192137 0.89205778]
Decoded qbit = [0.45192137 0.89205778]

MSE = 6.162975823700895e-33


## Verification

#### Single qubit errors

In [25]:
# verify for all single bit errors
for errType in range(3):
    for errIndex in range(n):

        # generate channel error
        errCheckRow = np.zeros(n, dtype = 'int')
        errCheckRow[errIndex] = 1
        if(errType == 0):
            errCheckRow = np.append(errCheckRow, errCheckRow) # Y error
        elif(errType == 1):
            errCheckRow = np.append(errCheckRow, np.zeros(n, dtype = 'int')) # X error
        else:
            errCheckRow = np.append(np.zeros(n, dtype = 'int'), errCheckRow) # Z error

        # apply channel error
        channel_error = getGenerator(errCheckRow)
        rx_erry = np.dot(channel_error, tx_encoded) # received qbits with error

        # get syndrome
        syndr = np.zeros([n-k, 1]) # syndrome
        for i in range(n-k):
            syndr[i] = np.dot(rx_erry.transpose(), np.dot(gi[i,:,:], rx_erry))

#         syndr[syndr>0] = 0
#         syndr[syndr<0] = 1
        
        # error correction
        recov = SyndromeLookUp(syndr) # error recovery
        rx_encoded = np.matmul(recov.transpose(), rx_erry) # received qbits without error but still encoded
        
        # setup optimizer to decode completely
        P = np.matmul(Gmatrix.transpose(), Gmatrix)
        q = -np.matmul(rx_encoded.transpose(), Gmatrix).flatten()
        x = cp.Variable(rx_encoded.shape[0])
        
        # solve for qubits numerically by minimizing distance
        prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, P) + q.T@x), [A@x == np.zeros(x.shape[0])])
        prob.solve()
        rx_decoded = (NormalizeState(x.value)) # received decoded qbits

        # print qubit errors
        error = tx_decoded - rx_decoded
        error = np.sum(error**2)/np.sum(np.abs(tx_decoded) > comparingAccuracy_decoded)
        print('Channel error = ' + str(errCheckRow) + ', MSE = ' + str(error))

Channel error = [1 0 0 0 0 1 0 0 0 0], MSE = 6.162975823700895e-33
Channel error = [0 1 0 0 0 0 1 0 0 0], MSE = 6.162975823700895e-33
Channel error = [0 0 1 0 0 0 0 1 0 0], MSE = 6.162975823700895e-33
Channel error = [0 0 0 1 0 0 0 0 1 0], MSE = 6.162975823700895e-33
Channel error = [0 0 0 0 1 0 0 0 0 1], MSE = 6.162975823700895e-33
Channel error = [1 0 0 0 0 0 0 0 0 0], MSE = 6.162975823700895e-33
Channel error = [0 1 0 0 0 0 0 0 0 0], MSE = 6.162975823700895e-33
Channel error = [0 0 1 0 0 0 0 0 0 0], MSE = 6.162975823700895e-33
Channel error = [0 0 0 1 0 0 0 0 0 0], MSE = 6.162975823700895e-33
Channel error = [0 0 0 0 1 0 0 0 0 0], MSE = 6.162975823700895e-33
Channel error = [0 0 0 0 0 1 0 0 0 0], MSE = 6.162975823700895e-33
Channel error = [0 0 0 0 0 0 1 0 0 0], MSE = 6.162975823700895e-33
Channel error = [0 0 0 0 0 0 0 1 0 0], MSE = 6.162975823700895e-33
Channel error = [0 0 0 0 0 0 0 0 1 0], MSE = 6.162975823700895e-33
Channel error = [0 0 0 0 0 0 0 0 0 1], MSE = 6.162975823700895

#### Two qubit errors

In [26]:
def analyzeChannel(errCheckRow):
    # apply channel error
    channel_error = getGenerator(errCheckRow)
    rx_erry = np.dot(channel_error, tx_encoded) # received qbits with error

    # get syndrome
    syndr = np.zeros([n-k, 1]) # syndrome
    for i in range(n-k):
        syndr[i] = np.dot(rx_erry.transpose(), np.dot(gi[i,:,:], rx_erry))

    # error correction
    recov = SyndromeLookUp(syndr) # error recovery
    rx_encoded = np.matmul(recov.transpose(), rx_erry) # received qbits without error but still encoded

    # setup optimizer to decode completely
    P = np.matmul(Gmatrix.transpose(), Gmatrix)
    q = -np.matmul(rx_encoded.transpose(), Gmatrix).flatten()
    x = cp.Variable(rx_encoded.shape[0])

    # solve for qubits numerically by minimizing distance
    prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, P) + q.T@x), [A@x == np.zeros(x.shape[0])])
    prob.solve()
    rx_decoded = (NormalizeState(x.value)) # received decoded qbits

    # print qubit errors
    if(np.sum(abs(tx_decoded) > comparingAccuracy_decoded) != np.sum(abs(rx_decoded) > comparingAccuracy_decoded)):
        print('Channel error = ' + str(errCheckRow) + ', skipped!')
        return
    error = tx_decoded - rx_decoded
    error = np.sum(error**2)/np.sum(np.abs(tx_decoded) > comparingAccuracy_decoded)
    print('Channel error = ' + str(errCheckRow) + ', MSE = ' + str(error))

In [27]:
myIndex1 = 1
string_format = '{:0>' + str(n) + '}' 
while(myIndex1 < 2**(n)):
    # generate weight 1 vectors
    checkRow1 = list(string_format.format("{:b}".format(myIndex1)))
    checkRow1 = np.asarray(list(map(int, checkRow1)))
    
    myIndex2 = myIndex1*2
    while(myIndex2 < 2**n):
        # generate another weight 1 vector
        checkRow2 = list(string_format.format("{:b}".format(myIndex2)))
        checkRow2 = np.asarray(list(map(int, checkRow2)))
        
        #generate weight 2 vector
        checkRow3 = list(string_format.format("{:b}".format(myIndex2)))
        checkRow3 = np.asarray(list(map(int, checkRow3)))
        checkRow3[checkRow1 == 1] = 1
        
        # add weight 2 errors with XX, XY, XZ, ...
        analyzeChannel(np.append(checkRow3, np.zeros(n, dtype = 'int')))
        analyzeChannel(np.append(checkRow3, checkRow1))
        analyzeChannel(np.append(checkRow2, checkRow1))
        
        analyzeChannel(np.append(checkRow3, checkRow2))
        analyzeChannel(np.append(checkRow3, checkRow3))
        analyzeChannel(np.append(checkRow2, checkRow3))
        
        analyzeChannel(np.append(checkRow1, checkRow2))
        analyzeChannel(np.append(checkRow1, checkRow3))      
        analyzeChannel(np.append(np.zeros(n, dtype = 'int'), checkRow3))
               
        myIndex2 = myIndex2*2
        
    myIndex1 = myIndex1*2   

Channel error = [0 0 0 1 1 0 0 0 0 0], MSE = 0.4084658472362666
Channel error = [0 0 0 1 1 0 0 0 0 1], MSE = 0.9999999999999998
Channel error = [0 0 0 1 0 0 0 0 0 1], MSE = 0.19372005722944335
Channel error = [0 0 0 1 1 0 0 0 1 0], MSE = 0.9999999999999998
Channel error = [0 0 0 1 1 0 0 0 1 1], MSE = 1.8062799427705567
Channel error = [0 0 0 1 0 0 0 0 1 1], MSE = 0.4084658472362666
Channel error = [0 0 0 0 1 0 0 0 1 0], MSE = 0.19372005722944335
Channel error = [0 0 0 0 1 0 0 0 1 1], MSE = 0.4084658472362666
Channel error = [0 0 0 0 0 0 0 0 1 1], MSE = 1.0000000000000002
Channel error = [0 0 1 0 1 0 0 0 0 0], MSE = 1.0000000000000002
Channel error = [0 0 1 0 1 0 0 0 0 1], MSE = 0.19372005722944335
Channel error = [0 0 1 0 0 0 0 0 0 1], MSE = 0.4084658472362666
Channel error = [0 0 1 0 1 0 0 1 0 0], MSE = 0.19372005722944335
Channel error = [0 0 1 0 1 0 0 1 0 1], MSE = 1.5915341527637334
Channel error = [0 0 1 0 0 0 0 1 0 1], MSE = 0.9999999999999998
Channel error = [0 0 0 0 1 0 0 1 0 0

### TODO
- MSE not good in 5 qbit code, why? Try Logical error rate
- WTF is Logical error rate? (or minimum fidelity)
- Problem in number of nonzero qbits not being equal in 5 qbit code, why?


- Change the channel error randomness to 1-3p,p,p,p
- Generate plots: mse (or some other rate) vs p


- Maybe start generating data

In [28]:
# print(tx_decoded.flatten())
# print(rx_decoded.flatten())

## Rough